# Import Required Libraries
Importa le librerie necessarie come mlflow, pandas e pyarrow.

In [ ]:
import mlflow
import pandas as pd

# Install and Import pyarrow
Installa pyarrow tramite %pip install pyarrow e importa pyarrow.parquet.

In [ ]:
%pip install pyarrow
import pyarrow.parquet as pq

# Read Parquet File and Convert to Pandas DataFrame
Leggi il file Parquet con pyarrow e converti la tabella in un DataFrame pandas.

In [ ]:
table = pq.read_table("/Volumes/main/fine_tuning_transformer_model/files/cleaned_CrifSept-OctCases.parquet")
pdf = table.to_pandas()

# Convert Pandas DataFrame to Spark DataFrame
Converte il DataFrame pandas in un DataFrame Spark usando spark.createDataFrame.

In [ ]:
df = spark.createDataFrame(pdf)
display(df)

# Select and Rename Columns
Seleziona le colonne 'Full Mail' e 'Level3 Category English Label', rinominandole rispettivamente in 'text' e 'label'.

In [ ]:
df = df.select("Full Mail", "Level3 Category English Label")\
    .withColumnRenamed("Full Mail", "text")\
    .withColumnRenamed("Level3 Category English Label", "label")
display(df)

# Load MLflow Model as Spark UDF
Carica il modello MLflow come Spark UDF usando mlflow.pyfunc.spark_udf e specifica il tipo di risultato.

In [ ]:
logged_model = "runs:/b5d0c759e6c74f9a903e5da7841dda61/classification"
classification_udf = mlflow.pyfunc.spark_udf(
    spark,
    model_uri=logged_model,
    result_type="string"
)

# Apply Model for Predictions
Applica la UDF di classificazione al DataFrame Spark per ottenere le predizioni.

In [ ]:
test = (
    df.limit(10)
    .select(
        df.text,
        df.label,
        classification_udf(df.text).alias("prediction")
    )
)

# Display Results
Visualizza i risultati delle predizioni usando display(test).

In [ ]:
display(test)